In [2]:
import numpy as np
import mediapipe as mp
import cv2


In [3]:
image = cv2.imread(r"C:/Users/pentest/Pictures/Camera Roll/demohand.jpg")


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands





#capture videofeed

cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands = 2,
    min_detection_confidence=0.5
) as hands:
    while cap.isOpened():
        success,image = cap.read()
        if not success:
            print("skip empty camera frame")
        else:
            #pass by reference to boost performance
            image.flags.writeable = True
            results = hands.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                    )
            
            cv2.imshow('Mediapipe Hands', cv2.flip(image,1))
            #wait for kill signal 
            if cv2.waitKey(5) and 0xFF == 27:
                break

cap.release()


    



   
    


KeyboardInterrupt: 

In [9]:
#setup mediapipe
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode



options = HandLandmarkerOptions(
    base_options = BaseOptions(model_asset_path="./hand_landmarker.task"),
    running_mode = VisionRunningMode.IMAGE
)



In [14]:
mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,data=image)

In [21]:
with HandLandmarker.create_from_options(options) as landmarker:
    landmarker_result = landmarker.detect(mp_image)
    print(landmarker_result)

HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
